## 七月在线机器学习实训营三期第二周(数据抓取)考试
#### 考试说明:
- 起止时间：请同学在2018年2月2日至2月4日期间完成，最晚提交时间本周日（2月4日24时之前）结束，<b>逾期不接受补考,该考试分数计入平时成绩</b>
- 考试方式：请同学<font color=red><b>拷贝</b></font>该试卷至自己姓名的目录后，将文件更名为同学姓名拼音-exam2后，进行作答。例如wangwei-exam2
- 提交格式：请同学新建自己姓名全拼的文件夹，将该试卷，zip文件等相关考试文件，放置此目录下。将该目录<b>移动</b>至/0.Teacher/Exam/2/目录下
- 注意事项：为确保同学们真正了解自身对本周课程的掌握程度，<font color=red><b>请勿翻阅，移动，更改</b></font>其它同学试卷。如发现按0分处理
- 请同学在下方同学姓名处填写自己的姓名，批改人和最终得分不用填写

- 同学姓名:<u>(王魁武)</u>  
- 批改人：林老师
- 最终得分: 75分

### 简答题 (共两题，每题5分)

#### 1.请使用BeautifulSoup定位到以下html代码段内容是“你好”的li标签？
```
<ul>
  <li class=eng>Hello</li>
  <li class=eng>World</li>
</ul>
<ul>
  <li class=chn>世界</li>
  <li class=chn>你好</li>
</ul>
```

In [12]:
from bs4 import BeautifulSoup
html='''
<ul>
  <li class=eng>Hello</li>
  <li class=eng>World</li>
</ul>
<ul>
  <li class=chn>世界</li>
  <li class=chn>你好</li>
</ul>
'''
bs=BeautifulSoup(html,'html5lib')
rows=bs.select('ul > li')
for i in rows:
    if i.text=='你好':
        sr1=i.text
print(sr1)

你好


#### 2.如果seleniun的find_xxx返回一个element，但是element.text获取文本为空，该如何进一步处理？

In [ ]:
if element:
    return element
else:
    element=element.value.split( )[i]
print(element)

# 林老师：用innerHTML获取含html标签的文本再过滤处理，-5分

### 代码题(共三题，每题30分)

#### 1.利用scrapy框架代码抓取https://sh.lianjia.com/ershoufang/changning/ 

前5页数据，   
 - 保留以下字段：信息链接、标题，单价、面积和总价。   
 - 去除面积50平方以下和单价9万以上。   
 - 数据以csv格式保存。   
 - 上传zip项目代码，确保scrapy crawl lianjia可以立即执行。

In [26]:
# -*- coding: utf-8 -*-
import scrapy

class LianjiaSpider(scrapy.Spider):
    name = 'lianjia'
    allowed_domains = ['lianjia.com']
    start_urls = ['https://sh.lianjia.com/ershoufang/changning/']
    pages=0

    def parse(self,response):
        text=response.text
        bs=BeautifulSoup(text,'html5lib')
        bs1=bs.select('ul.sellListContent > li')
        for i in bs1:
            product=i.selector('div.info clear')
            url=product.selector('div.title > a ').get('href')
            title=product.selector('div.title > a ').text.strip()
            area=product.selector('div.address > div').text.strip('|')[2]
            total_price=product.selector('div.priceInfo > div.totalPrice > span').text.strip()
            unit_price=product.selector('div.priceInfo > div.unitPrice >span').text.strip()
            yield{'url':url,
                  'title':title,
                  'unit_price':unit_price,
                  'area':area,
                  'total_price':total_price}
            LianjiaSpider.pages+=1
            if LianjiaSpider.parse>=5:
                return
            pages=bs.select('page-box house-lst-page-box > a')
            next_page=pages[-1]
            if next_page.text.strip()=='下一页':
                yield scrapy.Requsest(next_page.get('href').strip(),callback=self.parse)
                

# -*- coding: utf-8 -*-

# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: http://doc.scrapy.org/en/latest/topics/item-pipeline.html


class Data0204Pipeline(object):
    def process_item(self, item, spider):
        return item

class AreaFilter:
    def process_item(self,item,spider):
        if item and spider.name=='lianjia':
            if item['area']>=50:
                return item
        else:
            return item
        
class UnitPriceFilter:
    def prosess_item(self,item,spider):
        if item and spider.name=='lianjia':
            if item['unit_price']<90000:
                return item
        else:
            return item
        
import datetime as dt
import sqlite3 # 这里应该是from peewee import *，不然就要用默认的sqlite库操作，有区别的，-5分。

db=sqlite3.connect('lianjia.db')

class Product(model):
    url=TextField()
    title=TextField()
    unit_price=TextField()
    area=TextField()
    total=TextField()
    updated_at=DateTimeField(default=datetime.now())
    class Meta:
        database=db
        
class LianjiaSave:
    def open_spider(self,spider):
        if spider.name=='lianjia':
            db.create_tables([Product],safe=true)
            
    def process_item(self,item,spider):
        if item and spider.name=='lianjia':
            product=Product()
            product.url=item['url']
            product.title=item['title']
            product.unit_price=item['unit_price']
            product.area=item['area']
            product.total_price=item['total_price']
            

NameError: name 'model' is not defined

#### 2.使用scrapy实现豆瓣用户名密码登陆（包括验证码，手工输入登录信息）并获取豆瓣首页热门话题的标题和浏览人数。

In [27]:
# -*- coding: utf-8 -*-
import scrapy,urllib,re
from scrapy.http import Request,FormRequest

class DoubanSpider(scrapy.Spider):
    name='DouBan'
    allowed_domains =['douban.com']
    #start_urls = ['http://douban.com/']
    header={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 SE 2.X MetaSr 1.0'} #供登录模拟使用
    def start_requests(self):
        url='https://www.douban.com/accounts/login'
        return [Request(url=url,meta={'cookiejar':1},callback=self.parse)]

    def parse(self, response):
        text=response.text
        bs=BeautifulSoup(text,'html5lib')
        captcha=bs.select('img#captcha_image') #获取验证码图片的链接
        print(captcha)
        if captcha:
            captcha_value=input('请输入验证码：')
            data={
                "form_email": "328716930@qq.com",
                "form_password": "abcd1234",
                "captcha-solution": captcha_value,
            }
        else:
            print('无验证码')
            data={
                "form_email": "328716930@qq.com",
                "form_password": "abcd1234",
            }
    def get_content(self,response):
        text=response.text
        bs=BeautifulSoup(text,'html5lib')
        title=bs.select('div#db-nav-sns > div > div > div.nav-items > ul > li')
        if '我的豆瓣' in title:
            print('登录成功')
            hot_issue=bs.select('div.mod > ul > li.rec_topics')
            for i in hot_issue:
                print(i.selector('a').text.strip())
                print(i.selector('span').text.strip())
        else:
            print('登录失败')


#### 3.获取页面http://www.loewe.com/int/zh_CN/  女士系列/丝巾全部商品链接数据   
- （注意，必须展开页面）


In [ ]:
# 林老师，没有完成-15分。

### 本周课程意见反馈(10分送分项，非必答)
- 请同学围绕以下两点进行回答：
- 自身总结：请您对您自己在本周课程的学习，收获，技能掌握等方面进行一次总结 ，包括自身在哪些方面存在哪些不足，欠缺，困惑。作为将来回顾学习路径时的依据。
- 课程反馈：也可以就知识点，进度，难易度，教学方式，考试方式等等进行意见反馈，督促我们进行更有效的改进。


自身总结：先诉诉苦吧，作为零基础的同学，这周学起来蛮吃力的，收获不小，但差距更大，对爬虫掌握的还很不熟练，或者说只是有了个大体的概念吧，还处于入门级别吧，我觉得主要是因为没有一个系统性的认识，或许也是因为自己的基础太薄弱了吧。接下来只能不断的练习练习练习了！
说点浅见，如有不恰当的地方请见谅：我觉得老师一开始讲课的内容过于发散，我认为应该从基础开始讲一下比较好，先介绍框架，然后由某一点入手，再练成线，最后是面，最最后才是发散思维。一个循序渐进的系统性教程更为合适一些吧！